In [20]:
import geopandas as gpd
from gadm import GADMDownloader

import rasterio
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import datetime
import glob
import time

In [21]:
import pandas as pd
import numpy as np
import requests
import time
import os
import re
import warnings
import datetime 
import tempfile
import subprocess
import glob
import shutil
from itertools import product
import geopandas as gpd
from rasterstats import zonal_stats
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.transform import from_origin


In [22]:
from utils import (cross_df,
                month_start_day_to_month,
                pad2,
                pad3,
                file_to_raster,
                read_bm_csv,
                create_dataset_name_df,
                download_raster,
                define_variable,
                define_date_name,
                bm_extract_i,
                bm_raster_i)

In [23]:
downloader = GADMDownloader(version="4.0")

country_name = "Ghana"
ad_level = 1
roi_sf = downloader.get_shape_data_by_country_name(country_name=country_name, ad_level=ad_level)

In [24]:
product_id = 'VNP46A4'
date = 2021

if os.path.exists("/Users/robmarty/Desktop/bearer_bm.csv"):
    bearer = pd.read_csv("/Users/robmarty/Desktop/bearer_bm.csv")['token'][0]
else:
    bearer == "BEARER TOKEN HERE"

In [25]:

variable=None
quality_flag_rm = [255]
check_all_tiles_exist = True
output_location_type="tempfile"
file_dir=None
file_prefix=""
file_skip_if_exists=True
quiet = False
 
    
#### Make directory to put temporary files into
temp_main_dir = tempfile.gettempdir()

current_time_millis = int(round(time.time() * 1000))
temp_dir = os.path.join(temp_main_dir, "bm_raster_temp_" + str(current_time_millis))

os.makedirs(temp_dir)

#### Define NTL Variable
variable = define_variable(variable, product_id)

#### Ensure quality_flag_rm is a list
if type(quality_flag_rm) is not list:
    quality_flag_rm = [quality_flag_rm]

#### Ensure date is a list of strings
if type(date) is not list:
    date = [date]

date = [str(item) for item in date]

if file_dir is None:
    file_dir = os.getcwd()


In [26]:
date_i = "2021"

date_name_i = define_date_name(date_i, product_id)

date = date_i

In [27]:
#### Prep files to download

# Black marble grid: TODO: Add to python repo
bm_tiles_sf = gpd.read_file("https://raw.githubusercontent.com/worldbank/blackmarbler/main/data/blackmarbletiles.geojson")


# Prep dates            
if product_id == "VNP46A3":
    if len(date) <= 7:
        date = date + "-01"

if product_id == "VNP46A4":
    if len(date) in [4, 10]:
        date = date + "-01-01"


date = datetime.datetime.strptime(date, "%Y-%m-%d").date()

# Grab tile dataframe
year = date.year
month = date.month
day = date.timetuple().tm_yday

bm_files_df = create_dataset_name_df(product_id=product_id, all=True, years=year, months=month, days=day)

# Intersecting tiles
bm_tiles_sf = bm_tiles_sf[~bm_tiles_sf["TileID"].str.contains("h00")]
bm_tiles_sf = bm_tiles_sf[~bm_tiles_sf["TileID"].str.contains("v00")]

grid_use_sf = gpd.overlay(bm_tiles_sf, roi_sf.dissolve(), how='intersection')

# Make raster
tile_ids_rx = "|".join(grid_use_sf["TileID"])
bm_files_df = bm_files_df[bm_files_df["name"].str.contains(tile_ids_rx)]
bm_files_df = bm_files_df.reset_index()

#temp_dir = tempfile.gettempdir()

#shutil.rmtree(os.path.join(temp_dir, product_id), ignore_errors=True)

#### Create directory for tif files
shutil.rmtree(os.path.join(temp_dir, 'tif_files_tmp'), ignore_errors=True)
os.makedirs(os.path.join(temp_dir, 'tif_files_tmp'))




file_name = bm_files_df['name'][0]

n_tile = bm_files_df.shape[0] 

# Saves files in {temp_dir}/tif_files_tmp, which above is cleared and created
#download_raster(file_name, temp_dir, variable, bearer, quality_flag_rm, quiet, tile_i, n_tile)

   

In [43]:
# Path
year = file_name[9:13]
day = file_name[13:16]
product_id = file_name[0:7]

f = os.path.join(temp_dir, file_name) ##### CHANGEDDDDDD

'/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/bm_raster_temp_1698934131699/VNP46A4.A2021001.h17v07.001.2022094115525.h5'

In [18]:
wget_command = f"/usr/local/bin/wget -e robots=off -m -np .html,.tmp -nH --cut-dirs=3 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5000/{product_id}/{year}/{day}/{file_name}' --header 'Authorization: Bearer {bearer}' -P {temp_dir}/" 
subprocess.run(wget_command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Convert to raster
file_name_tif = re.sub(".h5", ".tif", file_name)

file_to_raster(f, variable, os.path.join(temp_dir, 'tif_files_tmp', file_name_tif), quality_flag_rm)

'/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/bm_raster_temp_1698933921458'

In [31]:
url = f'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5000/{product_id}/{year}/{day}/{file_name}'
headers = {'Authorization': f'Bearer {bearer}'}
download_path = os.path.join(temp_dir, file_name)

with httpx.stream('GET', url, headers=headers) as response:
    if response.status_code == 200:
        with open(download_path, 'wb') as file:
            for chunk in response.iter_bytes(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {file_name} to {download_path}")
    else:
        print(f"Failed to download {file_name}. Status code: {response.status_code}")


Downloaded VNP46A4.A2021001.h17v07.001.2022094115525.h5 to /var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/bm_raster_temp_1698934131699/VNP46A4.A2021001.h17v07.001.2022094115525.h5


In [44]:
temp_dir

'/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/bm_raster_temp_1698934131699'

In [45]:
file_name_tif = re.sub(".h5", ".tif", file_name)

In [46]:
f

'/var/folders/m1/8h14xfm56hd6qfgz6btm1rd80000gn/T/bm_raster_temp_1698934131699/VNP46A4.A2021001.h17v07.001.2022094115525.h5'

In [47]:
file_to_raster(f, variable, os.path.join(temp_dir, 'tif_files_tmp', file_name_tif), quality_flag_rm)

In [ ]:


try:
    #### Create raster for each date
    raster_path_list = []    
    for date_i in date:

        date_name_i = define_date_name(date_i, product_id)

        raster_path_i = bm_raster_i(roi_sf, product_id, 
                                    date_i, bearer, 
                                    variable, quality_flag_rm, check_all_tiles_exist, quiet, temp_dir)
        raster_path_list.append(raster_path_i) 

    #### Stack Rasters
    # Read the first file to get the dimensions and metadata
    with rasterio.open(raster_path_list[0]) as src:
        width = src.width
        height = src.height
        count = len(raster_path_list)
        crs = src.crs
        transform = src.transform
        dtype = src.dtypes[0]

        # Create an empty numpy array to store the raster data
        data = np.zeros((count, height, width), dtype=dtype)

        # Read data from each file and store it in the numpy array
        for i, file_path in enumerate(raster_path_list):
            with rasterio.open(file_path) as src:
                data[i] = src.read(1)  

    # Create a new raster file and write the data
    timestamp = str(int(time.time()))
    tmp_raster_file_name = product_id + "_" + date[0].replace("-", "_") + "_" + timestamp + ".tif"

    with rasterio.open(os.path.join(temp_dir, 
                                    tmp_raster_file_name), 
                       'w', driver='GTiff', width=width, height=height,
                       count=count, dtype=dtype, crs=crs, transform=transform) as dst:
        for i in range(count):
            dst.write(data[i], i+1) 

    # Move from black marble temp folder to main temp folder
    shutil.move(os.path.join(temp_dir, tmp_raster_file_name), 
                temp_main_dir) 

    r_out = rasterio.open(os.path.join(temp_main_dir, tmp_raster_file_name))

except:
    # Delete temp files used to make raster
    shutil.rmtree(temp_dir, ignore_errors=True)

    if quiet == False:
        print("Skipping " + str(date_i) + " due to error. Data may not be available.\n")
    r_out = None

# Delete temp files used to make raster
shutil.rmtree(temp_dir, ignore_errors=True)
